# From DRY to Design Patterns

## i. Repeated Code Detection

Inside the directory `src/main/java/dungeonmania/entities/enemies`, there is occurrence of repeated code, particularly concerning the movement logic found in different enemy classes such as `Mercenary` and `ZombieToast`. This logic revolves around generating a random subsequent position for the enemy to progress to and then executing the move. And code for defining how to move when consuming invincibility potion

logic for move randomly
```java
Position nextPos;
GameMap map = game.getMap();
List<Position> pos = getPosition().getCardinallyAdjacentPositions();
pos = pos
    .stream()
    .filter(p -> map.canMoveTo(this, p)).collect(Collectors.toList());
if (pos.size() == 0) {
    nextPos = getPosition();
    game.getMap().moveTo(this, nextPos);
} else {
    nextPos = pos.get(randGen.nextInt(pos.size()));
    game.getMap().moveTo(this, nextPos);
}
```

and logic observed for move when invincible

```java
else if (map.getPlayer().getEffectivePotion() instanceof InvincibilityPotion) {
            Position plrDiff = Position.calculatePositionBetween(map.getPlayer().getPosition(), getPosition());

            Position moveX = (plrDiff.getX() >= 0) ? Position.translateBy(getPosition(), Direction.RIGHT)
                    : Position.translateBy(getPosition(), Direction.LEFT);
            Position moveY = (plrDiff.getY() >= 0) ? Position.translateBy(getPosition(), Direction.UP)
                    : Position.translateBy(getPosition(), Direction.DOWN);
            Position offset = getPosition();
            if (plrDiff.getY() == 0 && map.canMoveTo(this, moveX))
                offset = moveX;
            else if (plrDiff.getX() == 0 && map.canMoveTo(this, moveY))
                offset = moveY;
            else if (Math.abs(plrDiff.getX()) >= Math.abs(plrDiff.getY())) {
                if (map.canMoveTo(this, moveX))
                    offset = moveX;
                else if (map.canMoveTo(this, moveY))
                    offset = moveY;
                else
                    offset = getPosition();
            } else {
                if (map.canMoveTo(this, moveY))
                    offset = moveY;
                else if (map.canMoveTo(this, moveX))
                    offset = moveX;
                else
                    offset = getPosition();
            }
            nextPos = offset;
```

## ii. Suitable Design Pattern

The **Template Method Pattern** is an ideal design pattern to address the code repetition observed in the `Mercenary` and `ZombieToast` classes. This pattern allows us to define a skeleton of an algorithm in a method, deferring some steps to subclasses. It lets subclasses redefine certain steps of an algorithm without changing the algorithm's structure.

In the context of our code, the movement logic for different enemy types has common steps but varies in some aspects. For instance, the logic to move randomly and to move when invincible is common across different enemy types, but the specifics of how the next position is determined can vary.

By applying the Template Method Pattern, we can move the common logic to the `Enemy` superclass and allow subclasses to provide their own implementation for determining the next position. This way, the `moveRandom` and `moveWhenInvincible` methods can be defined in the `Enemy` class, and each subclass like `Mercenary` and `ZombieToast` can override the `determineNextPosition` method to provide their own specific logic.

This approach not only reduces code duplication but also makes the code more flexible and easier to maintain. Changes to the common logic need to be made in one place only, and it's easy to add new enemy types with different movement logic.


## iii. Refactoring the code:

In our efforts to reduce repetition and maintain clarity in the code, we applied design principles and techniques that address the observed issues.

The repetitive logic for random enemy movement and behavior during player invincibility was identified in the classes `Mercenary` and other enemy types. To address this, we introduced new methods in the superclass `Enemy`:

1. **moveRandom**: This method captures the logic for the enemy to move in a random direction among the available valid positions. By centralizing this logic in the superclass, subclasses like `Mercenary` can now invoke this method without repeating the logic.

```java
public void moveRandom(Game game) {
    ...
}
```

2. **moveWhenInvincible**: The behavior of enemies when the player consumes an `InvincibilityPotion` was also consistent among enemy types. This method was introduced to centralize this logic.

```java
public void moveWhenInvincible(Game game) {
    ...
}
```



With these methods in place in the superclass, the `Mercenary` and `ZombieToast` class were then refactored to use these new methods. We focused on reducing nesting and improving readability:

1. **move**: We refactoed this method in Mercenary.java for the enemy's movement. Here, based on various conditions like player-alliance, player invisibility, or invincibility, appropriate movement strategies are selected.

```java
   @Override
    public void move(Game game) {
        Position nextPos;
        GameMap map = game.getMap();

        if (allied) {
            nextPos = moveWhenAllied(game);
        } else if (map.getPlayer().getEffectivePotion() instanceof InvisibilityPotion) {
            moveWhenInvisible(game);
            return;
        } else if (map.getPlayer().getEffectivePotion() instanceof InvincibilityPotion) {
            moveWhenInvincible(game);
            return;

        } else {
            nextPos = moveNormally(game);
        }

        map.moveTo(this, nextPos);
    }
```

Similarly ZombieToast.java move method was refactored:

```java
    @Override
    public void move(Game game) {
        GameMap map = game.getMap();

        if (map.getPlayer().getEffectivePotion() instanceof InvincibilityPotion) {
            moveWhenInvincible(game);
        } else {
            moveRandom(game);
        }
    }
```

Through these refactorings, we ensured a clear separation of concerns, reduced code duplication, and enhanced readability. This makes the system more maintainable, extensible, and comprehensible.


## Blog Post: Refactoring to the Open-Closed Principle

### Analysis of Current Design

#### i. Evaluation of Design Quality and Open-Closed Principle Compliance
The current implementation of `Goal` employs a `switch` statement to handle various types of goals (`exit`, `boulders`, `treasure`, `AND`, `OR`). This design raises some flags regarding maintainability and scalability:

- **Adding a new goal type requires modifying the existing `Goal` class**, which could introduce bugs into the system.
- **The `switch` statement grows larger and more complex** with each new goal type, making the code harder to understand and maintain.
- **Repeated logic across different cases** suggests a potential violation of the DRY (Don't Repeat Yourself) principle.

Given these points, it's clear that the current design does not fully comply with the open-closed principle.

### Refactoring with Design Patterns

#### ii. Choosing a Design Pattern
To improve the code quality and adhere to the open-closed principle, we can apply the **Composite Pattern**. This pattern is ideal for situations where you need to represent a part-whole hierarchy of objects. Here's why it fits our scenario:

- **Individual Goal Types**: Each goal type can be treated as an individual object with common operations.
- **Tree Structure**: The `AND` and `OR` goals naturally form a tree structure, which is a perfect use case for the Composite Pattern.
- **Extensibility**: Adding new goal types becomes as simple as creating a new class that implements the `Goal` interface.

### Implementation of Refactor

#### Refactoring Steps
1. **Create a `Goal` Interface**: Define common operations like `achieved()` and `toString()`.
2. **Implement Concrete Goal Classes**: Develop classes like `AndGoal`, `BouldersGoal`, `ExitGoal`, `OrGoal`, and `TreasureGoal` that implement the `Goal` interface.
3. **Update `GoalFactory`**: Utilize these concrete classes to instantiate goal objects, eliminating the `switch` statement.
4. **Remove the Old `Goal` Class**: Discard the class that uses the `switch` statement, which is no longer needed.

#### Advantages of the New Design
- **Maintainability**: The new design makes the code easier to maintain since changes to one goal type do not affect others.
- **Scalability**: Adding new goal types is now a matter of adding a new class.
- **Testability**: Each goal type can be tested in isolation.

### Conclusion

By refactoring the `Goal` class to use the Composite Pattern, we've significantly improved our code's maintainability and scalability. We've transformed a rigid structure into a flexible and extensible system that respects the Open-Closed Principle. This sets a solid foundation for future development and is more robust.

# Implementing the Enemies Goal: A Microevolution in Game Design

## Requirements Engineering

Upon reviewing the task requirements, we recognized the need to track the number of enemies defeated and check for the presence of spawners. Assumptions were made regarding the types of enemies and spawners, deciding to focus on a generic enemy counter and specifically on `ZombieToastSpawner` for spawner checks.

## Detailed Design

To meet the new goal specifications, the following additions were planned:

- **`EnemiesGoal` Class**: A new class within the `goals` package to encapsulate the logic of the enemy goal.
  - **Fields**: `enemyTarget` to store the number of enemies to defeat.
  - **Methods**: `achieved(Game game)` to determine if the goal has been met, and `areAllSpawnersGone(Game game)` to check spawner existence.
- **Integration**: Modify `GoalFactory` to recognize and instantiate `EnemiesGoal` objects based on configuration input.

## Design Review

A peer review session was conducted to validate the design. Feedback was positive, with minor suggestions to ensure `EnemiesGoal` conformed to the existing `Goal` interface for seamless integration.

## Test List

Our test list focused on:

- Verifying that the goal is not achieved prematurely.
- Checking that the goal is achieved when the enemy count is met and spawners are destroyed.

## Test List Review

The test list was reviewed by a teammate, confirming comprehensive coverage and logical test case construction.

## Create the Skeleton

A skeleton for the `EnemiesGoal` class and test cases were created, laying the groundwork for development and ensuring a TDD approach.

## Development

Functionality was implemented incrementally, with tests passing at each stage:

1. **Enemy Count Tracking**: `enemiesDestroyedCount` was added to `Game.java` to keep track of defeated enemies.
2. **`EnemiesGoal` Implementation**: The class was created and integrated with the rest of the goal system.
3. **`GoalFactory` Update**: Added logic to create `EnemiesGoal` based on configuration.
4. **Testing**: Tests were written first, failing initially, and subsequently passed as functionality was added.


# Implementing Sun Stone and New Buildables in Dungeon Mania

## Introduction
In this update, we've expanded Dungeon Mania with the introduction of a new collectible entity, the Sun Stone, and two new buildable entities, the Sceptre and Midnight Armour. These additions come with unique functionalities and have been integrated into the game's mechanics and objectives.

## Requirements Engineering
The Sun Stone can open doors and substitute for crafting materials, but with the unique trait of being retained after use. The Sceptre gives players the power to control mercenaries, while the Midnight Armour provides a permanent boost to the player's stats.

## Design

### New Classes and Packages
- `dungeonmania.entities.collectables.SunStone`: Handles the Sun Stone's collectability and its use in opening doors and crafting.
- `dungeonmania.entities.buildables.Sceptre`: Defines the Sceptre, its duration, and the immediate effect upon use.
- `dungeonmania.entities.buildables.MidnightArmour`: Encapsulates the properties of Midnight Armour, including its permanent attack and defence bonuses.
- `dungeonmania.entities.collectables.Collectable`: New interface to standardize how collectable items are engaged by the player.
- `dungeonmania.entities.buildables.BuildableFactory`: A factory class that encapsulates the creation of different buildable entities based on a set of properties.

### Modifications to Existing Classes
- `dungeonmania.entities.inventory.Inventory`: Updated to handle the new logic for crafting with the Sun Stone.
- `dungeonmania.entities.Entity`: Modified to accommodate interactions with the new collectable and buildable entities.
- `dungeonmania.entities.Player`: Enhanced with methods to interact with collectables and to determine if zombies are present.
- `dungeonmania.map.GameMap`: Now checks for the presence of zombies, affecting the crafting of Midnight Armour.
- `dungeonmania.entities.Door`: Modifed door to work to open with sunstone
- `dungeonmania.entities.enemies.Mercenary`: Modifed mercanary to be allied when we use sceptre. 

## Design Review
The design was peer-reviewed and iterated upon to ensure cohesiveness and compatibility with the existing codebase. Special attention was given to the interaction between new entities and the game's rules.

## Test List for Sun Stone and More Buildables

Before implementing the new features outlined in our design, we've established a comprehensive list of test cases to ensure our code meets the functional requirements. Below, we provide an overview of the tests we will conduct.

### SunStone Tests

1. **Picking Up SunStone**: To confirm that a player can pick up a SunStone when moving over it. This test checks if the SunStone appears in the player's inventory after the movement.

2. **Using SunStone To Open Door**: This test ensures that a player can use the SunStone to open doors. We'll verify the door's status before and after the interaction, and ensure the SunStone remains in the inventory post-use.

3. **Treasure Goal With SunStone**: To validate that possessing a SunStone contributes towards fulfilling a treasure collection goal within the game.

### Buildables Tests

1. **Building a Sceptre with SunStone**: We will assess whether a Sceptre can be crafted using a SunStone in place of a treasure or key. The inventory should reflect the creation of the Sceptre and the consumption of the ingredients.

2. **Building a Shield with SunStone**: This test verifies that a Shield can be built with a SunStone, and ensures the SunStone is retained in the inventory after crafting.

3. **Midnight Armour Crafting Restrictions**: To ensure that Midnight Armour cannot be crafted if zombies are present in the dungeon. The inventory should remain unchanged after the failed crafting attempt.

4. **Midnight Armour Crafting without Zombies**: Conversely, this test checks that Midnight Armour can be crafted when no zombies are present, and that the ingredients are consumed in the process.

5. **Sceptre's Influence on Mercenary**: We will verify that a player with a Sceptre can interact with a mercenary to ally them without engaging in combat.

### Test Review
We had the test cases reviewed by a peer to ensure we've covered all necessary scenarios and that our tests are logically sound and complete.

## Create the Skeleton
We stubbed out new classes and methods, ensuring that they conform to the planned design and are ready for implementation.

## Development and Testing
We followed test-driven development (TDD) practices, writing tests first, which initially failed, and then implemented the functionality to pass the tests.

